# Séance 6 : Map/Reduce avec PIG

Plan

- [Exercice 1 : GROUP BY](#exo1)
- [Exercice 2 : JOIN](#exo2)
- [Prolongements](#prol)

<div style="position:absolute; top:10px; right:5px; width:100px; margin:10px; background-color:#DDDDDD;">
[Exo 1 : GROUP BY](#exo1) -- [Exo 2 : JOIN](#exo2) -- [Prolongements](#prol)
</div>

On considère le jeu de données suivant : [Localization Data for Person Activity Data Set](https://archive.ics.uci.edu/ml/datasets/Localization+Data+for+Person+Activity) qu'on récupère comme suit :

In [1]:
import pyquickhelper, pyensae
params={"server":"", "username":"dz...fr", "password":""}
pyquickhelper.open_html_form(params=params,title="server + credentials", key_save="params")

In [4]:
password = params["password"]
server = params["server"]
username = params["username"]

In [5]:
%remote_open

<h3 id="exo1">Exercice 1 : GROUP BY</h3>

In [9]:
import pandas, sqlite3
con = sqlite3.connect("ConfLongDemo_JSI.db3")
df = pandas.read_sql("""SELECT activity, count(*) as nb FROM person GROUP BY activity""", con)
con.close()
df.head()

,activity,nb
0,falling,2973
1,lying,54480
2,lying down,6168
3,on all fours,5210
4,sitting,27244


Il faut maintenant le faire avec PIG.

In [8]:
%%PIG solution_groupby.pig

myinput = LOAD 'ConfLongDemo_JSI.small.example.txt' 
          using PigStorage(',') 
          AS (index:long, sequence, tag, timestamp:long, dateformat, x:double,y:double, z:double, activity) ;

gr = GROUP myinput BY activity ;
avgact = FOREACH gr GENERATE group, COUNT(myinput) ; 

STORE avgact INTO 'ConfLongDemo_JSI.small.group.txt' USING PigStorage() ;

In [9]:
%jobsubmit solution_groupby.pig groupby.redirection

In [19]:
%remote_cmd tail groupby.redirection.err

In [22]:
%remote_cmd hdfs dfs -ls ConfLongDemo_JSI.small.group.txt

In [23]:
%remote_cmd hdfs dfs -tail ConfLongDemo_JSI.small.group.txt/part-r-00000

<h3 id="exo2">Exercice 2 : JOIN</h3>

In [10]:
con = sqlite3.connect("ConfLongDemo_JSI.db3")
df = pandas.read_sql("""SELECT person.*, A.nb FROM person INNER JOIN (
                            SELECT activity, count(*) as nb FROM person GROUP BY activity) AS A
                            ON person.activity == A.activity""", con)
con.close()
df.head()

,index,sequence,tag,timestamp,dateformat,x,y,z,activity,nb
0,0,A01,010-000-024-033,633790226051280329,27.05.2009 14:03:25:127,4.062931,1.892434,0.507425,walking,32710
1,1,A01,020-000-033-111,633790226051820913,27.05.2009 14:03:25:183,4.291954,1.781140,1.344495,walking,32710
2,2,A01,020-000-032-221,633790226052091205,27.05.2009 14:03:25:210,4.359101,1.826456,0.968821,walking,32710
3,3,A01,010-000-024-033,633790226052361498,27.05.2009 14:03:25:237,4.087835,1.879999,0.466983,walking,32710
4,4,A01,010-000-030-096,633790226052631792,27.05.2009 14:03:25:263,4.324462,2.072460,0.488065,walking,32710


Idem, maintenant il faut le faire avec PIG.

In [27]:
%%PIG solution_groupby_join.pig

myinput = LOAD 'ConfLongDemo_JSI.small.example.txt' 
          using PigStorage(',') 
          AS (index:long, sequence, tag, timestamp:long, dateformat, x:double,y:double, z:double, activity) ;

gr = GROUP myinput BY activity ;
avgact = FOREACH gr GENERATE group, COUNT(myinput) ; 

joined = JOIN myinput BY activity, avgact BY group ;

STORE joined INTO 'ConfLongDemo_JSI.small.group.join.txt' USING PigStorage() ;

In [28]:
%jobsubmit solution_groupby_join.pig groupby.join.redirection

In [35]:
%remote_cmd tail groupby.join.redirection.err

In [37]:
%remote_cmd hdfs dfs -ls ConfLongDemo_JSI.small.group.join.txt

In [39]:
%remote_cmd hdfs dfs -tail ConfLongDemo_JSI.small.group.join.txt/part-r-00000

<h3 id="prol">Prolongements</h3>

[PIG](http://pig.apache.org/) n'est pas la seule façon d'exécuter des jobs Map/Reduce. [Hive](https://hive.apache.org/) est un langage dont la syntaxe est très proche de celle du SQL. L'article [Comparing Pig Latin and SQL for Constructing Data Processing Pipelines](https://developer.yahoo.com/blogs/hadoop/comparing-pig-latin-sql-constructing-data-processing-pipelines-444.html) explicite les différences des deux approches.

**langage haut niveau**

Ce qu'il faut retenir est que le langage PIG est un langage haut niveau. Le programme est compilé en une séquence d'opérations Map/Reduce transparente pour l'utilisateur. Le temps de développement est très réduit lorsqu'on le compare au même programme écrit en Java. Le compilateur construit un plan d'exécution ([quelques exemples ici](http://chimera.labs.oreilly.com/books/1234000001811/ch07.html#explain)) et infère le nombre de machines requises pour distribuer le job. Cela suffit pour la plupart des besoins, cela nécessite.

**petits jeux**

Certains jobs peuvent durer des heures, il est conseillée de les essayer sur des petits jeux de données avant de les faire tourner sur les vrais données. Il est toujours frustrant de s'apercevoir qu'un job a planté au bout de deux heures car une chaîne de caractères est vide et que ce cas n'a pas été prévu.

Avec ces petits jeux, il est possible de faire tourner et conseillé de tester le job d'abord sur la passerelle ([exécution local](http://archive.cloudera.com/cdh/3/pig/tutorial.html#Running+the+Pig+Scripts+in+Local+Mode)) avant de le lancer sur le cluster. Avec pyensae, il faut ajouter l'option ``-local`` à la commande ``jobsubmit``.

**concaténer les fichiers divisés**

Un programme PIG ne produit pas un fichier mais plusieurs fichiers dans un répertoire. La commande [getmerge](http://hadoop.apache.org/docs/r2.3.0/hadoop-project-dist/hadoop-common/FileSystemShell.html) télécharge ces fichiers sur la passerelle et les fusionne en un seul.

**ordre des lignes**

Les jobs sont distribués, même en faisant rien (LOAD + STORE), il n'est pas garanti que l'ordre des lignes soit préservé. La probabilié que ce soit le cas est quasi nulle.